In [1]:
using Luxor

In [2]:
using JSON

In [3]:
using DataFrames
using DataFramesMeta
using CSV

In [4]:
pwd()

"/home/martin/Science/southampton/runs/mediterranean"

In [5]:
scale_x(x) = x * 500 + 10
scale_y(y) = y * 500 + 10
scale(x, y) = scale_x(x), scale_y(y)

scale (generic function with 1 method)

In [62]:
function draw_link(l, cities, risk, count, draw_unused=false)
#    if link_type(l) == "fast"
#        sethue("green")
#        w = 10 * (1-width)
#    else
#        sethue("red")
#        w = (15 - width)/2
#    end

    if count == 0.05
        w = 0.5
        sethue(risk*20, 1-risk*20, 0)
        setline(w)
        setdash("dashed")
    else
        w = count * 10
        sethue(risk*20, 1-risk*20, 0)
        setline(w)
        setdash("solid")
    end
    
    p1, p2 = lcoords(l, cities)
    if count > 0.05 || draw_unused
        line(Point(scale(p1...)...), Point(scale(p2...)...), :stroke)
    end
end

function draw_city(c, size, name)
    setdash("solid")
    if city_type(c) == "exit"
        sethue("blue")
    else
        sethue("purple")
    end
    x, y = scale(coords(c)...)
    circle(Point(x, y), size * 5, :fill)
    sethue("black")
    setline(1)
    circle(Point(x, y), 5, :stroke)
    text(name, Point(x+6, y+2))
end

draw_city (generic function with 2 methods)

In [43]:
coords(c :: DataFrameRow) = c[:y], 1.0-c[:x]

coords (generic function with 1 method)

In [44]:
function lcoords(link :: DataFrameRow, cities :: DataFrame)
    c1 = cities[cities.id .== link[:l1], :][1, :]
    c2 = cities[cities.id .== link[:l2], :][1, :]
    
    coords(c1), coords(c2)
end
    

lcoords (generic function with 1 method)

In [45]:
link_type(l :: DataFrameRow) = lowercase(l[:type])
city_type(c :: DataFrameRow) = lowercase(c[:type])

city_type (generic function with 1 method)

In [54]:
function draw_landscape_j(dir, name = "output"; risk=true, draw_unused=false, use_opt=false)
    suffix = use_opt ? "_opt" : ""
    cities = CSV.read(dir * "/cities$suffix.txt", delim="\t", ignorerepeated=true, header=1, copycols=true, DataFrame)
    links = CSV.read(dir * "/links$suffix.txt", delim="\t", ignorerepeated=true, header=1, copycols=true, DataFrame)

    map = JSON.parsefile("map_med1.json")
    
    sort!(links, :count)
    
    desc_c = describe(cities)
    max_cc = desc_c[(desc_c.variable .== :count), :max][1]
    desc_l = describe(links)
    max_cl = desc_l[(desc_l.variable .== :count), :max][1]
    
    @pdf begin

        origin(0,0)
        background("white")

        setline(1)
        box(Point(scale(-0.01, -0.01)...), Point(scale(1.01, 1.01)...), :stroke)
        
#        line(Point())

        
        
        for l in eachrow(links)
            count_f = l[:count] / max_cl * 0.95 + 0.05
            if risk
                draw_link(l, cities, l[:risk], count_f, draw_unused) 
            else
                draw_link(l, cities, l[:friction]/3, count_f, draw_unused) 
            end
        end
        
        #q = 
        
        fontsize(5)
        
        for c in eachrow(cities)
            name = find_by_coord(c[:x], c[:y], map)
            s = c[:count]
            draw_city(c, s / max_cc, name)
        end

    end 520 520 name * ".pdf"
end

draw_landscape_j (generic function with 2 methods)

In [63]:
draw_landscape_j("done/x9_99_1_6030321324244463672/", "map_med", risk=true, draw_unused=true)


In [38]:
    map = JSON.parsefile("map_med1.json")


Dict{String, Any} with 3 entries:
  "links"         => Any[Dict{String, Any}("distance"=>0.0194039, "j"=>2, "risk…
  "needs entries" => true
  "cities"        => Any[Dict{String, Any}("geotype"=>"C", "name"=>"Sabratah", …

In [40]:
map["cities"][1]

Dict{String, Any} with 6 entries:
  "geotype" => "C"
  "name"    => "Sabratah"
  "typ"     => 1
  "x"       => 0.529087
  "country" => "Libya"
  "y"       => 0.0878283

In [41]:
closeby(x1, y1, x2, y2, eps) = sqrt((x1-x2)^2 + (y1-y2)^2) < eps

function find_by_coord(x, y, map)
    cities = map["cities"]
    for c in cities
        if closeby(x, y, c["x"], c["y"], 0.0001)
            return c["name"]
        end
    end
    return ""
end

find_by_coord (generic function with 1 method)